In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import scanpy as sc
import scvi

ModuleNotFoundError: No module named 'scvi'

In [ ]:
sc.pp.normalize_total(source_adata)
sc.pp.log1p(source_adata)
source_adata.raw = source_adata # keep full dimension safe
sc.pp.highly_variable_genes(
    source_adata,
    n_top_genes=5000,
    batch_key="author",
    layer="counts",
    subset=True)

In [ ]:
scvi.model.SCANVI.setup_anndata(adata, layer = "counts", batch_key="author", labels_key="cell_type")

In [ ]:
early_stopping_kwargs = {
    "early_stopping_metric": "elbo",
    "save_best_state_metric": "elbo",
    "patience": 10,
    "threshold": 0,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}
early_stopping_kwargs_scanvi = {
    "early_stopping_metric": "accuracy",
    "save_best_state_metric": "accuracy",
    "on": "full_dataset",
    "patience": 10,
    "threshold": 0.001,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}
early_stopping_kwargs_surgery = {
    "early_stopping_metric": "elbo",
    "save_best_state_metric": "elbo",
    "on": "full_dataset",
    "patience": 10,
    "threshold": 0.001,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}

In [ ]:
model = scvi.models.SCANVI(
    adata,
    "Unknown",
    n_layers=2,
    encode_covariates=True,
    deeply_inject_covariates=False,
    use_layer_norm="both",
    use_batch_norm="none",
    use_cuda = True # indicate to use gpu!
)

In [ ]:
model.train(
    n_epochs_unsupervised=500,
    n_epochs_semisupervised=200,
    unsupervised_trainer_kwargs=dict(early_stopping_kwargs=early_stopping_kwargs),
    semisupervised_trainer_kwargs=dict(metrics_to_monitor=["elbo", "accuracy"],
                                       early_stopping_kwargs=early_stopping_kwargs_scanvi),
    frequency=1
)

In [ ]:

adata.obsm["X_scVI"] = vae.get_latent_representation()
adata.obs["pred_label"] = vae.predict()